# Tools calling with LangGraph React Agent

In [ ]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

In [ ]:
from devtools import debug
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool

from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt


In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

# llm = EdenAI(
#     feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
# )
llm = get_llm("gpt_35_azure")
#llm = get_llm("gpt_35_openai")

# Define lit of tools

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"], custom_vocabulary=None, speakers=None),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]

###  Select and execute and Eden.AI provided tool with React agents

In [ ]:
input_ = """i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input_})

In [ ]:
debug(response)

###  Same with Tool Calling Agent

###  Select and execute and Eden.AI provided tool with React agents

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)



response = agent_executor.invoke({"input": input_})

In [ ]:
debug(response)

###  Same with LangGraph React Implementation (actually using tools)

Does not work yet with eden api

In [ ]:
from langgraph.prebuilt import create_react_agent

# langgraph "create_react_agent"  is similar to langchain "create_tool_calling_agent"AgentExecutor"AgentExecutor"AgentExecutor
# Don't confuse with legacy lanchain "create_react_agent"


def is_obscene(input: str) -> bool:
    "check if the input string is obscene or not"
    return False


# DOES NOT WORK with Edenai tools...

# tools = [is_obscene]  # This work !

graph = create_react_agent(llm, tools=tools)

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", input_)]}
print_stream(graph.stream(inputs, stream_mode="values"))